In [3]:
from pymongo import MongoClient
import hashlib
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from pymongo.errors import DuplicateKeyError
import random
import string
from pymongo.errors import OperationFailure


In [4]:
class App:
    def __init__(self, master):
        self.master = master
        master.title("Product Management")

        # Style
        self.style = ttk.Style()
        self.style.configure('TButton', font=('calibri', 10, 'bold'), foreground='black')

        # Buttons
        self.insert_button = ttk.Button(master, text="Insert", command=self.prepare_insert)
        self.insert_button.grid(row=0, column=0, padx=10, pady=5)

        self.update_button = ttk.Button(master, text="Update", command=self.update)
        self.update_button.grid(row=0, column=1, padx=10, pady=5)

        self.delete_button = ttk.Button(master, text="Delete", command=self.prepare_delete)
        self.delete_button.grid(row=0, column=2, padx=10, pady=5)

        self.read_button = ttk.Button(master, text="Read", command=self.read)
        self.read_button.grid(row=0, column=3, padx=10, pady=5)

    def prepare_insert(self):
        self.insert()

    def insert(self):
        # Create a new window for data input
        insert_window = tk.Toplevel(self.master)
        insert_window.title("Insert Product Data")

        # ASIN label and entry field
        tk.Label(insert_window, text="ASIN:").grid(row=0, column=0)
        self.asin_entry = tk.Entry(insert_window)
        self.asin_entry.grid(row=0, column=1)

        # Generate ASIN button
        generate_asin_button = ttk.Button(insert_window, text="Generate ASIN", command=self.generate_asin)
        generate_asin_button.grid(row=0, column=2)

        # Other entry fields
        tk.Label(insert_window, text="Price:").grid(row=1, column=0)
        self.price_entry = tk.Entry(insert_window)
        self.price_entry.grid(row=1, column=1)

        tk.Label(insert_window, text="Feature:").grid(row=2, column=0)
        self.feature_entry = tk.Entry(insert_window)
        self.feature_entry.grid(row=2, column=1)

        tk.Label(insert_window, text="Title:").grid(row=3, column=0)
        self.title_entry = tk.Entry(insert_window)
        self.title_entry.grid(row=3, column=1)

        tk.Label(insert_window, text="Rank:").grid(row=4, column=0)
        self.rank_entry = tk.Entry(insert_window)
        self.rank_entry.grid(row=4, column=1)

        tk.Label(insert_window, text="Brand:").grid(row=5, column=0)
        self.brand_entry = tk.Entry(insert_window)
        self.brand_entry.grid(row=5, column=1)

        tk.Label(insert_window, text="Image URL:").grid(row=6, column=0)
        self.image_url_entry = tk.Entry(insert_window)
        self.image_url_entry.grid(row=6, column=1)

        # Insert button
        insert_button = ttk.Button(insert_window, text="Insert", command=self.insert_data)
        insert_button.grid(row=7, columnspan=3, pady=10)

    def generate_asin(self):
        # Generate a random ASIN
        asin = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        # Set the generated ASIN in the ASIN entry field
        self.asin_entry.delete(0, tk.END)
        self.asin_entry.insert(0, asin)

    def prepare_delete(self):
        # Create a new window for data input
        delete_window = tk.Toplevel(self.master)
        delete_window.title("Delete Product Data")

        # ASIN label and entry field
        tk.Label(delete_window, text="ASIN:").grid(row=0, column=0)
        self.delete_asin_entry = tk.Entry(delete_window)
        self.delete_asin_entry.grid(row=0, column=1)

        # Delete button
        delete_button = ttk.Button(delete_window, text="Delete", command=self.delete_data)
        delete_button.grid(row=1, columnspan=2, pady=10)

    def delete_data(self):
        # Get ASIN to delete
        asin = self.delete_asin_entry.get()
        shard_index = complex_hash(asin)
        client = MongoClient('mongodb://localhost:27017/')

        # collection name list
        db_list = ['db_1', 'db_2', 'db_3']

        # Create a new database and collection
        db_backup = client['db_backup']
        collection_backup = db_backup['collection_backup']

        db = client[db_list[shard_index]]
        collection = db['collection']
        if asin:
            # Delete data from MongoDB based on ASIN ID
            result = collection.delete_one({"_id": asin})
            if result.deleted_count == 0:
                messagebox.showinfo("Delete", f"No data with ASIN: {asin} was found.")
            else:
                messagebox.showinfo("Delete", f"Data with ASIN: {asin} deleted successfully.")

            result_backup = collection_backup.delete_one({"_id": asin})
            if result_backup.deleted_count == 0:
                messagebox.showinfo("Delete", f"No data with ASIN: {asin} in backup was found.")
            else:
                messagebox.showinfo("Delete", f"Data with ASIN: {asin} from backup deleted successfully.")
        else:
            messagebox.showwarning("Delete", "Please provide an ASIN to delete.")

    def insert_data(self):
        # Get data from entry fields
        asin = self.asin_entry.get()
        price = self.price_entry.get()
        feature = self.feature_entry.get()
        title = self.title_entry.get()
        rank = self.rank_entry.get()
        brand = self.brand_entry.get()
        image_url = self.image_url_entry.get()

        # Hash ASIN to determine the shard index
        shard_index = complex_hash(asin)
        client = MongoClient('mongodb://localhost:27017/')

        # collection name list
        db_list = ['db_1', 'db_2', 'db_3']

        # Create a new database and collection
        db_backup = client['db_backup']
        collection_backup = db_backup['collection_backup']

        db = client[db_list[shard_index]]
        collection = db['collection']
        # Create product data dictionary
        product_data = {
            "_id": asin,
            "price": float(price),
            "feature": feature,
            "title": title,
            "rank": int(rank),
            "brand": brand,
            "imageURL": image_url
        }

        # Insert data into MongoDB
        try:
            collection.insert_one(product_data)
            messagebox.showinfo("Insert", "Data inserted successfully.")
        except DuplicateKeyError:
            messagebox.showerror("Insert Error", "ASIN already exists in the database.")

        try:
            collection_backup.insert_one(product_data)
            messagebox.showinfo("Insert", "Data inserted successfully.")
        except DuplicateKeyError:
            messagebox.showerror("Insert Error", "ASIN already exists in the database.")


    def update(self):
            # Create a new window for updating data
            update_window = tk.Toplevel(self.master)
            update_window.title("Update Product Data")

            # ASIN label and entry field
            tk.Label(update_window, text="ASIN:").grid(row=0, column=0)
            self.update_asin_entry = tk.Entry(update_window)
            self.update_asin_entry.grid(row=0, column=1)

            # Update button
            update_button = ttk.Button(update_window, text="Update Data", command=self.update_data_window)
            update_button.grid(row=1, columnspan=2, pady=10)

    def update_data_window(self):
        # Retrieve ASIN from entry field
        asin = self.update_asin_entry.get()
        if asin:
            # Now add entry fields for updating title, brand, imageURL, feature, price, rank
            # Add corresponding labels and entry fields here...
            product_info = get_product_info(asin)

            if product_info:
                # Create a new window for updating data
                update_data_window = tk.Toplevel(self.master)
                update_data_window.title("Update Product Data")

                # Title label and entry field
                tk.Label(update_data_window, text="Title:").grid(row=0, column=0)
                self.update_title_entry = tk.Entry(update_data_window)
                self.update_title_entry.insert(0, product_info['title'])
                self.update_title_entry.grid(row=0, column=1)

                # Brand label and entry field
                tk.Label(update_data_window, text="Brand:").grid(row=1, column=0)
                self.update_brand_entry = tk.Entry(update_data_window)
                self.update_brand_entry.insert(0, product_info['brand'])
                self.update_brand_entry.grid(row=1, column=1)

                # Image URL label and entry field
                tk.Label(update_data_window, text="Image URL:").grid(row=2, column=0)
                self.update_image_url_entry = tk.Entry(update_data_window)
                self.update_image_url_entry.insert(0, product_info['imageURL'])
                self.update_image_url_entry.grid(row=2, column=1)

                # Feature label and entry field
                tk.Label(update_data_window, text="Feature:").grid(row=3, column=0)
                self.update_feature_entry = tk.Entry(update_data_window)
                self.update_feature_entry.insert(0, product_info['feature'])
                self.update_feature_entry.grid(row=3, column=1)

                # Price label and entry field
                tk.Label(update_data_window, text="Price:").grid(row=4, column=0)
                self.update_price_entry = tk.Entry(update_data_window)
                self.update_price_entry.insert(0, product_info['price'])
                self.update_price_entry.grid(row=4, column=1)

                # Rank label and entry field
                tk.Label(update_data_window, text="Rank:").grid(row=5, column=0)
                self.update_rank_entry = tk.Entry(update_data_window)
                self.update_rank_entry.insert(0, product_info['rank'])
                self.update_rank_entry.grid(row=5, column=1)
            else:
                messagebox.showerror("Update Error", "ASIN not found.")
            # Create Update button
            update_button = ttk.Button(update_data_window, text="Update data to MongoDB", command=lambda: self.update_data(asin))
            update_button.grid(row=6, columnspan=2, pady=10)    


        else:
            messagebox.showwarning("Update", "Please provide an ASIN.")


    def update_data(self, asin):
        # Retrieve entered values
        title = self.update_title_entry.get()
        brand = self.update_brand_entry.get()
        image_url = self.update_image_url_entry.get()
        feature = self.update_feature_entry.get()
        price = float(self.update_price_entry.get())
        rank = int(self.update_rank_entry.get())

        # Update data in MongoDB
        try:
            # Connect to MongoDB
            client = MongoClient('mongodb://localhost:27017/')
            
            # Select database and collection
            shard_index = complex_hash(asin)
            db_list = ['db_1', 'db_2', 'db_3']
            db = client[db_list[shard_index]]
            collection = db['collection']

            # Construct update query
            update_query = {
                "$set": {
                    "title": title,
                    "brand": brand,
                    "imageURL": image_url,
                    "feature": feature,
                    "price": price,
                    "rank": rank
                }
            }

            # Update document based on ASIN
            result = collection.update_one({"_id": asin}, update_query)

            if result.modified_count == 1:
                messagebox.showinfo("Update", f"Data with ASIN: {asin} updated successfully.")
            else:
                messagebox.showerror("Update Error", "Failed to update data.")

        except Exception as e:
            messagebox.showerror("Update Error", f"An error occurred: {str(e)}")

        try:
            # Connect to MongoDB
            client = MongoClient('mongodb://localhost:27017/')
            
            # Select database and collection
            db_backup = client['db_backup']
            collection_backup = db_backup['collection_backup']

            # Construct update query
            update_query = {
                "$set": {
                    "title": title,
                    "brand": brand,
                    "imageURL": image_url,
                    "feature": feature,
                    "price": price,
                    "rank": rank
                }
            }

            # Update document based on ASIN
            result = collection_backup.update_one({"_id": asin}, update_query)

            if result.modified_count == 1:
                messagebox.showinfo("Update", f"Data with ASIN: {asin} updated successfully.")
            else:
                messagebox.showerror("Update Error", "Failed to update data.")

        except Exception as e:
            messagebox.showerror("Update Error", f"An error occurred: {str(e)}")



    def read(self):
        read_window = tk.Toplevel(self.master)
        read_window.title("Read Product Data")

        # ASIN label and entry field
        tk.Label(read_window, text="ASIN:").grid(row=0, column=0)
        self.read_asin_entry = tk.Entry(read_window)
        self.read_asin_entry.grid(row=0, column=1)

        # Display button
        display_button = ttk.Button(read_window, text="Display Info", command=self.display_info)
        display_button.grid(row=1, columnspan=2, pady=10)

    def display_info(self):
        # Get ASIN to read
        asin = self.read_asin_entry.get()
        if asin:
            # Retrieve product information based on ASIN ID
            product_info = get_product_info(asin)
            if product_info:
                # Display product information
                info_window = tk.Toplevel(self.master)
                info_window.title("Product Info")

                # Configure font style
                font_style = ('Arial', 12)  # Change the font family and size as needed

                # Wrap length for labels
                wrap_length = 400  # Adjust as needed based on your window size

                # Display product information with larger and nicer font
                tk.Label(info_window, text=f"ASIN: {product_info['_id']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Price: {product_info['price']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Feature: {product_info['feature']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Title: {product_info['title']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Rank: {product_info['rank']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Brand: {product_info['brand']}", font=font_style, wraplength=wrap_length).pack()
                tk.Label(info_window, text=f"Image URL: {product_info['imageURL']}", font=font_style, wraplength=wrap_length).pack()
            else:
                messagebox.showerror("Read Error", f"ASIN: {asin} not found.")
        else:
            messagebox.showwarning("Read", "Please provide an ASIN to read.")



def complex_hash(asin, num_shards=3):
    # Use SHA-256 hash function to hash the ASIN
    hash_object = hashlib.sha256(asin.encode())
    # Get the hexadecimal representation of the hash
    hex_dig = hash_object.hexdigest()
    # Convert the hex digest to an integer
    int_hash = int(hex_dig, 16)
    # Use the modulo operation to get an index for the shard
    shard_index = int_hash % num_shards
    return shard_index

def get_product_info(asin):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    shard_index = complex_hash(asin)
    # Select database and collection
    db_list = ['db_1', 'db_2', 'db_3']

    db = client[db_list[shard_index]]
    collection = db['collection']

    # Query for the document based on ASIN ID
    product_info = collection.find_one({"_id": asin})

    return product_info

root = tk.Tk()
app = App(root)
root.mainloop()


2024-04-04 16:58:26.245 python[32006:15060562] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2024-04-04 16:58:27.556 python[32006:15060562] Warning: Expected min height of view: (<NSButton: 0x7f8e7f5b8890>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-04-04 16:58:34.481 python[32006:15060562] Warning: Expected min height of view: (<NSButton: 0x7f8e6eba3680>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-04-04 16:58:38.355 python[32006:15060562] Warning: Expected min height of view: (<NSButton: 0x7f8e6ebc5ed0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
